In [ ]:
import json
from pathlib import Path
import openai
import re
import math
import numpy as np
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import time

from utils import extract_numbers,find_first_empty_index,process_scientific_notation,extract_numeric_answer
openai.api_key  = "OPENAI_API_KEY"

from sentence_transformers import SentenceTransformer

model_sentence = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# Download APBench_Gamma.json from woodywu/APBench 
filename = 'APBench_Gamma.json'
data = json.loads(Path(filename).read_text())
# Print the tenth problem and the 1st question among the questions
index, order = 10, 1
content   = data[index]['Content']
question_solution_so_on = data[index]['Questions'][order]
print("Question:", question_solution_so_on['Question'])
print("Solution:", question_solution_so_on['Solution'])
print("Answer:", question_solution_so_on['Answer'])
print("Format:", question_solution_so_on['Format'])
print("Source:", question_solution_so_on['Source'])

In [3]:
def get_embedding(text):
    return model_sentence.encode(text)

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

### Example question

In [4]:
msg = '''The Viking Mars orbiter probe was placed in a circular orbit 17,000 km above the Martian surface. Doppler measurements of the transmitted signals from the probe indicated that it was at an orbital velocity, \( V_V \), of 1.46 km/s. The diameter of Mars, \( d_M \), is 6,770 km and the radius of the Mars orbit about the Sun, \( a_M \), is 1.524 AU. Note: 1 AU = 1.495 × 10^8 km; 1 year = 3.156 × 10^7 s. Calculate the mass of Mars, \( M_M \), in terms of the mass of the Sun, \( M_S \). Without any extra words, please specify the answer in the final sentence with this form: 'the answer is (answer)'. The answer should either be numeric or symbolic (not words).'''

In [ ]:
# model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-3.1-8B-Instruct"
model_id = "meta-llama/Llama-3.1-70B-Instruct"
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = "meta-llama/Llama-3.2-1B-Instruct"

# 4-bit quant
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # Automatically map layers across available GPUs
    torch_dtype=torch.bfloat16,  # Use bf16 precision
    quantization_config={"load_in_4bit": True},  # 4-bit quantization
)

# Set up the pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)



Loading checkpoint shards: 100%|██████████| 30/30 [00:27<00:00,  1.11it/s]


In [10]:
start_time = time.time()
for i in range(1):
    messages = [
        {"role": "user", "content": msg}
    ]
    generated_text = pipe(
        messages,
        max_length = 2024
    )
    reply = generated_text[0]['generated_text'][-1]['content']
    print(reply)
print(f'Elapsed time: {(time.time()-start_time)/1}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


To calculate the mass of Mars, \( M_M \), we'll first calculate the orbital period of the Viking Mars orbiter, then use it to find the mass of Mars. 

Given:
\( V_V \) = 1.46 km/s = 1460 m/s
\( r \) = 17,000 km + (6,770 km / 2) = 20,385 km = 20,385,000 m (orbital radius)
\( a_M \) = 1.524 AU = 1.524 * 1.495 × 10^8 km = 2.279 × 10^8 km = 2.279 × 10^11 m (average distance from Sun)

First, calculate the orbital period, \( T \), of the Viking Mars orbiter:

\( V_V \) = \( 2 \pi r \) / \( T \)
\( T \) = \( 2 \pi r \) / \( V_V \)
\( T \) = \( 2 \pi \) * 20,385,000 m / 1460 m/s
\( T \) = 88,431 s

Now, calculate the mass of Mars using the formula for the orbital velocity:

\( V_V \) = \( \sqrt{ G M_M / r } \)
\( M_M \) = \( r V_V^2 \) / \( G \)

Using the gravitational constant \( G \) = 6.674 × 10^-11 N m^2 kg^-2 and the previously calculated values for \( r \) and \( V_V \):

\( M_M \) = (20,385,000 m) * (1460 m/s)^2 / (6.674 × 10^-11 N m^2 kg^-2)
\( M_M \) = 6.418 × 10^23 kg

Now, calcula

In [10]:
data[6]

{'Content': 'The Juno spacecraft arrived at Jupiter on July 4, 2016, to begin an examination of the planet’s gravity field and magnetosphere from an orbit that will bring it very close each passage. For this question, you may assume the answer from each previous part even if you did not derive the previous answer. You must show your work clearly for full points. Some facts:\n- \\( R_J = 71,500 \\) km,\n- \\( a_J = 5.2025870 \\) AU,\n- \\( GM_J = 1.26686 \\times 10^{8} \\) km\\(^3\\)/s\\(^2\\),\n- \\( GM_{\\odot} = 1.32712438 \\times 10^{11} \\) km\\(^3\\)/s\\(^2\\).',
 'Questions': [{'Question': '(a) Assume a lowest energy Hohmann transfer ellipse and show that Juno encountered Jupiter with a relative speed of \\( v_{\\infty} = 5.7 \\) km/s, taking the heliocentric orbits of Jupiter and Earth to be circular and in the same orbital plane. Please provide the answer in the unit of kilometers per second (km/s).',
   'Solution': "(a) From Kepler and Newton,\n\\[ v^2 = GM_\\odot \\left( \\fr

In [22]:
match_result = []  # Universal array of results

for idx in range(len(data)):#range(65): #range(45):#range(7): #
    # idx = 2
    print("_____________")
    print("ID", idx)
    print("current_match_result: ", match_result)


    content_str = data[idx]['Content']
    questions_solutions = data[idx]['Questions']
    question_strs = [entry['Question'] for entry in questions_solutions]
    solution_strs = [entry['Solution'] for entry in questions_solutions]
    answer_strs = [entry['Answer'] for entry in questions_solutions]
    format_strs = [entry['Format'] for entry in questions_solutions]
    match_results_local = [[] for entry in questions_solutions]

    # print("content: ",content_str)

    correct_QAs = []
    for q_idx, question_str in enumerate(question_strs):
        print("______")
        print("q_idx: ", q_idx)
        format_str = format_strs[q_idx]
        solution_str = solution_strs[q_idx]
        answer_str = answer_strs[q_idx]
        print("Answer: ", answer_str)
        # all_correct_QAs = " ".join(correct_QAs)
        all_correct_QAs = " " # not appending option
        msg = f"{all_correct_QAs} {content_str} {question_str} Without any extra words, please specify the answer in the final sentence with this form: 'the answer is (answer)'. The answer should either be numeric or symbolic (not words)."

        print("msg", msg)
        
        # calling model
        messages = [
            {"role": "user", "content": msg}
        ]
        generated_text = pipe(
            messages,
            max_length = 2024
        )
        reply = generated_text[0]['generated_text'][-1]['content']
        print(reply)
        
        pattern = r'the answer is \s*(.*)|The answer is \s*(.*)' 
        match = re.search(pattern, reply)
        try:
            user_answer_str = match.group(0)
            print("user_answer_str: ", user_answer_str)
        except:
            user_answer_str = None

        if 'message' == format_str:
            print("MESSAGE")

            score = 0

            # embedding score
            score_embedding = cosine_similarity(get_embedding(reply), get_embedding(answer_str))
            print(f"similarity: {score_embedding}")
            
            # gpt score
            # gpt prompt
            message = f"""
            Compare the following reply with the expected answer and evaluate their alignment using these criteria:

            1. **Relevance:** Does the reply address the scientific goals and concepts mentioned in the expected answer?  
            2. **Completeness:** Does the reply cover the key points about the need for a deep periapsis and a large range of distances?  
            3. **Accuracy:** Is the reasoning in the reply correct and consistent with the scientific explanation?  

            Question: {question_str}  
            Expected Answer: {answer_str}  
            Reply: {reply}  

            After evaluating, provide a similarity score between 0 and 10, where:  
            - **10** means the reply perfectly aligns with the expected answer.  
            - **0** means the reply does not align at all.  

            Only return the numeric score as your final output.
            """

            # response part
            response = openai.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": message}],
                temperature=0,
            )
            gpt_similarity = response.choices[0].message.content
            try:
                score_gpt = float(gpt_similarity) / 10 
            except ValueError:
                score_gpt = 0

            # combining scores
            score = score_embedding*0.5 + score_gpt*0.5
            print(f"score_gpt: {score_gpt}")
            print(f"SCORE: {score}")
            
            if score > 0.6: # more than 60% affirmative
                match_results_local[q_idx] = 1
                extra_str = "For reference, a previous question: '" + content_str + question_str + " '  has answer " + reply
                correct_QAs.append(extra_str)
            else:
                match_results_local[q_idx] = 0 

            ## formated output
            print(f'LLM answer: {reply}')
            print(f'Reference answer: {answer_str}')
            print('True' if match_results_local[q_idx]==1 else 'False')
           
        if 'numeric' == format_str:
            print("NUMERIC")

            power = []
            try:
                if user_answer_str is None:
                    user_answer_str = ''
                    raise TypeError("user_answer_str cannot be None")
                user_answer_num_str = extract_numeric_answer(user_answer_str)
                user_answer_num = extract_numbers(user_answer_num_str)[0]
            except:
                # check if answer string has power expression
                user_answer_str, power = process_scientific_notation(user_answer_str)
                user_answer_num = None

            # when openai is able to handle the operation
            if not power:
                user_answer_num = user_answer_num
                answer_str_num = extract_numbers(answer_str)[0]
                match_results_local[q_idx] = 0
            # other cases
            else:
                try:
                    if power:
                        print("pow")
                        user_answer_pow = extract_numbers(user_answer_str.strip())[2]
                        user_answer_num = extract_numbers(user_answer_str.strip())[0] * 10**user_answer_pow
                        power = extract_numbers(answer_str)[1]
                        answer_str_num = extract_numbers(answer_str)[0] * 10**power

                        print("user_answer_num", user_answer_num)
                        print("answer_str_num", answer_str_num)

                    else:
                        print("no pow")
                        print(user_answer_str.strip())
                        user_answer_num = extract_numbers(user_answer_str.strip())[0]
                        answer_str_num = extract_numbers(answer_str)[0]

                        print("user_answer_num", user_answer_num)
                        print("answer_str_num", answer_str_num)

                except:
                    user_answer_num = None
                    answer_str_num = None
                    match_results_local[q_idx] = 0

            if user_answer_num is not None and user_answer_num != 0:
                
                margin = 0.1 + 0.01 * math.log(abs(user_answer_num))
                print("margin: ", margin)
                if answer_str_num == user_answer_num:
                    match_results_local[q_idx] = 1
                    extra_str = "For reference, a previous question: " + question_str + ". It has answer " + str(user_answer_num)
                    correct_QAs.append(extra_str)
                    # print("Exact Match")
                elif abs(answer_str_num - user_answer_num) / abs(user_answer_num) < abs(margin):
                    match_results_local[q_idx] = 1
                    extra_str = "For reference, a previous question: " + question_str + ". It has answer " + str(user_answer_num)
                    correct_QAs.append(extra_str)
                    # print("Rough Match")
                else:
                    match_results_local[q_idx] = 0
                    # print("No Match")
            
            ## formated output
            print(f'LLM answer: {user_answer_str}')
            print(f'Reference answer: {answer_str}')
            print('True' if match_results_local[q_idx]==1 else 'False')

    if match_results_local:
        match_results_local_new = [item for item in match_results_local if not (isinstance(item, list) and len(item) == 0)]
        print("local", match_results_local_new)
        match_result.extend(match_results_local_new)

# Print the universal match result array
print("Universal match result array:", match_result)


total_accuracy = sum(match_result) / len(match_result)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


_____________
ID 0
current_match_result:  []
______
q_idx:  0
Answer:  The J2 gravitational perturbation causes a change in the right ascension of the ascending node (\(\Omega\)) as described by the equation above.
msg   Let us use Lagrange’s planetary equations to compute how the J2 gravitational perturbation will affect the orbit elements. The disturbance potential function \( R(r) \) for the J2 oblateness component of the spherical gravitational harmonics is defined in Eq. (11.56) and is given by

\[
R(r) = -\frac{J_2}{2}\frac{\mu}{r}\left(\frac{r_{eq}}{r}\right)^2\left(3\sin^2\phi - 1\right)
\]

Using Eq. (9.145), we note that 

\[
\sin \phi = \hat{r}\cdot \hat{z} = \sin(\omega + f)\sin i
\]

This allows us to write the disturbance potential function \( R \) in terms of the chosen orbit elements as

\[
R(e) = -\frac{J_2}{2}\frac{\mu}{r}\left(\frac{r_{eq}}{r}\right)^2\left(3\sin^2(\omega + f)\sin^2 i - 1\right) \tag{12.85}
\]

Expressing the orbit radius using Eq. (9.6) and performi

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


score_gpt: 1.0
SCORE: 0.7114536762237549
LLM answer: \[
\frac{d\Omega}{dt} = -\frac{3}{2}J_2n\left(\frac{r_{eq}}{p}\right)^2\cos i
\]

the answer is \(-\frac{3}{2}J_2n\left(\frac{r_{eq}}{p}\right)^2\cos i\).
Reference answer: The J2 gravitational perturbation causes a change in the right ascension of the ascending node (\(\Omega\)) as described by the equation above.
True
______
q_idx:  1
Answer:  The J2 gravitational perturbation causes a change in the inclination (\(i\)) as described by the equation above.
msg   Let us use Lagrange’s planetary equations to compute how the J2 gravitational perturbation will affect the orbit elements. The disturbance potential function \( R(r) \) for the J2 oblateness component of the spherical gravitational harmonics is defined in Eq. (11.56) and is given by

\[
R(r) = -\frac{J_2}{2}\frac{\mu}{r}\left(\frac{r_{eq}}{r}\right)^2\left(3\sin^2\phi - 1\right)
\]

Using Eq. (9.145), we note that 

\[
\sin \phi = \hat{r}\cdot \hat{z} = \sin(\omega + f)\sin i

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

margin:  0.14202451303547953
LLM answer: the answer is 66.85°
Reference answer: 135.088 degrees
False
local [0, 0, 0]
_____________
ID 17
current_match_result:  [1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
______
q_idx:  0
Answer:  350 degrees
msg   If the DCM for the transformation from \( xyz \) to \( x'y'z' \) is:

\[
[Q] = \begin{bmatrix}
0.64050 & 0.75319 & -0.15038 \\
0.76736 & -0.63531 & 0.086824 \\
-0.030154 & -0.17101 & -0.98481
\end{bmatrix}
\] What is the angle \(\alpha\) in the classical Euler sequence? Please provide the answer in degrees. Without any extra words, please specify the answer in the final sentence with this form: 'the answer is (answer)'. The answer should either be numeric or symbolic (not words).
To find the angle \(\alpha\) in the classical Euler sequence, we need to find the rotation angle

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


margin:  0.12278292400425002
LLM answer: the answer is 9.76.
Reference answer: 350 degrees
False
______
q_idx:  1
Answer:  170.0 degrees
msg   If the DCM for the transformation from \( xyz \) to \( x'y'z' \) is:

\[
[Q] = \begin{bmatrix}
0.64050 & 0.75319 & -0.15038 \\
0.76736 & -0.63531 & 0.086824 \\
-0.030154 & -0.17101 & -0.98481
\end{bmatrix}
\] What is the angle \(\beta\) in the classical Euler sequence? Please provide the answer in degrees. Without any extra words, please specify the answer in the final sentence with this form: 'the answer is (answer)'. The answer should either be numeric or symbolic (not words).


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


To find angle \(\beta\) in the classical Euler sequence, we first need to identify the rotation matrix for the rotation around the y-axis by 90 degrees (since the classical Euler sequence is \( \alpha \), \( \beta \) around y, \( \gamma \) ) and isolate the corresponding rotation matrix from the DCM. 

However, given the DCM, we can also find \( \beta \) directly from the elements of the DCM. 

The rotation matrix for \( \beta \) around y is:
\[
R_y(\beta) = \begin{bmatrix}
cos(\beta) & 0 & sin(\beta) \\
0 & 1 & 0 \\
-sin(\beta) & 0 & cos(\beta)
\end{bmatrix}
\]

Given the DCM, we can see that the \( R_y(\beta) \) elements are:
\[
cos(\beta) = 0.64050 \\
sin(\beta) = -0.030154 \\
cos(\beta) = 0.98481
\]

However, \( cos(\beta) \) appears in two different places, but they are inconsistent, and this DCM is not a rotation matrix for \( \beta \) around y only. It is likely a combination of multiple rotations. 

In this case, we can only find \( \beta \) from the most consistent elements of

In [24]:
print(total_accuracy)

0.32432432432432434
